# Identify which SPT flares have coinciding observations in TESS
This notebook searches whether could have been looking at the SPT flares by time TESS and identifies such sectors.
The identifying data of the flares from this notebook are saved to `spt_tess_candidates.pkl` and `spt_flare_sources.pkl`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

import lightkurve as lk
from tess_stars2px import tess_stars2px_function_entry as tesspoint

import pickle

## First we will retrieve the SPT flare details
The minimum information we need for selecting TESS sectors is (ra,dec,mjd)

The SPT Flare Star Catalog (2019-2022) is retrieved from https://pole.uchicago.edu/public/data/tandoi24/#Overview.

Current versions of `spt_tess_candidates.pkl` and `spt_flare_sources.pkl` contain data from `'../data/SPT/2023_june2025_winter_stars_for_TESS.csv'`, `'../data/SPT/online_pipeline_flagged_stars_12jun2025.csv'`, `'../data/SPT/spt3g_2019_2022_flare_star_catalog.csv'`

In [43]:
filepath= '../data/SPT/online_pipeline_flagged_stars_12jun2025.csv'
spt_flares_df = pd.read_csv(filepath)
print(spt_flares_df.columns)

Index(['ra', 'dec', 'obsid', 'mjd'], dtype='object')


In [44]:
# Replace the following with whatever column indicates flare datetime
# Ideally the dateime is flare peak if given, otherwise SPT single scan start works too
flare_datetime_col_name = 'mjd'

# Replace the following with whatever column indicates flare (ra,dec)
# Ideally the use Gaia DR3 source_ra and source_dec for consistency
ra_col_name = 'ra'
dec_col_name = 'dec'

print(spt_flares_df[flare_datetime_col_name].head())

0    59338.648877
1    59430.214259
2    59447.555035
3    59456.828588
4    59464.056574
Name: mjd, dtype: float64


In [45]:
# Convert flare_time to utc and mjd
spt_flares_df['flare_time_mjd'] = spt_flares_df[flare_datetime_col_name]#.apply(lambda x: Time(x, scale= 'utc', format= 'iso').utc.mjd)
spt_flares_df.sort_values(by= 'flare_time_mjd')
spt_flares_df['source_sectors'] = [None for _ in range(len(spt_flares_df))]
spt_flares_df['flare_sector'] = [None for _ in range(len(spt_flares_df))]
print(spt_flares_df.columns)

Index(['ra', 'dec', 'obsid', 'mjd', 'flare_time_mjd', 'source_sectors',
       'flare_sector'],
      dtype='object')


## Identify coinciding TESS sectors
To shorten the list of possible sectors, we will take advantage of the [TESS-point package](https://github.com/tessgi/tess-point) which gives the list of TESS Sectors that look at a given (ra, dec). <span style="color:red; font-weight:bold;">See [TESS-point Web Tool](https://github.com/tessgi/tess-point) as well. </span>

Then we search by sector datetime intervals retrieved from https://tess.mit.edu/observations/ to identify the sector(s) that overlap with SPT's observation.

But first we have to process the TESS sector data.

In [46]:
TESS_sectors_df = pd.read_csv("..\data\TESS\TESS_orbit_times.csv")

# Times are stored by orbit, so first merge by sector
TESS_sectors_df = TESS_sectors_df.groupby("Sector").agg({
    "Start of Orbit": "min",
    "End of Orbit": "max"
}).reset_index()
TESS_sectors_df.columns = ["Sector", "Sector Start", "Sector End"]

# Formatting to MJD for comparison
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df['Sector'] = TESS_sectors_df['Sector'].apply(lambda x: int(x))
TESS_sectors_df['Sector Start'] = TESS_sectors_df['Sector Start'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
TESS_sectors_df['Sector End'] = TESS_sectors_df['Sector End'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
print(TESS_sectors_df.columns)

Index(['Sector', 'Sector Start', 'Sector End'], dtype='object')


Now we narrow the list of sectors

In [47]:
for i, flare in spt_flares_df.iterrows():
    ra = flare[ra_col_name]
    dec = flare[dec_col_name]
    flare_time = flare['flare_time_mjd']
    # c = SkyCoord(ra, dec, unit= 'deg')

    # Search for any sectors where TESS is looking at the star
    # possible_matches = lk.search_tesscut(c, [sector for sector in valid_sectors])
    outID, outEclipLong, outEclipLat, outSec, outCam, _, _, _, scinfo = tesspoint(i, ra, dec)

    flare['source_sectors'] = outSec
    spt_flares_df.at[i, 'source_sectors'] = outSec

    if len(outSec) >= 1:
        print(f'{i}/{spt_flares_df.shape[0]}: {outSec}')

        # Check time range for each sector
        for sector in outSec:
            sector_time = TESS_sectors_df[TESS_sectors_df['Sector'] == sector]
            if sector_time.empty:
                print(f'\tOrbit times for sector {sector} and beyond are not available in TESS_orbit_times.csv. Likely indicates source is to be observed in the future. TESS_orbit_times.csv should be updated when available.')
                break
            sector_start = sector_time['Sector Start'].iloc[0]
            sector_end = sector_time['Sector End'].iloc[0]
            if sector_start <= flare_time <= sector_end:
                print(f'\tFlare observed in Sector {sector}')
                spt_flares_df.at[i, 'flare_sector'] = int(sector)
            else:
                print(f'\tSector {sector} doesn\'t overlap with flare')
    else:
        print(f'{i}/{spt_flares_df.shape[0]}: TESS doesn\'t look at target')


0/39: [  1   2  28  29  68  69  95  96 103 104]
	Sector 1 doesn't overlap with flare
	Sector 2 doesn't overlap with flare
	Sector 28 doesn't overlap with flare
	Sector 29 doesn't overlap with flare
	Sector 68 doesn't overlap with flare
	Sector 69 doesn't overlap with flare
	Sector 95 doesn't overlap with flare
	Orbit times for sector 96 and beyond are not available in TESS_orbit_times.csv. Likely indicates source is to be observed in the future. TESS_orbit_times.csv should be updated when available.
1/39: [  1  27  67  94  95 101 102 103 104]
	Sector 1 doesn't overlap with flare
	Sector 27 doesn't overlap with flare
	Sector 67 doesn't overlap with flare
	Sector 94 doesn't overlap with flare
	Sector 95 doesn't overlap with flare
	Orbit times for sector 101 and beyond are not available in TESS_orbit_times.csv. Likely indicates source is to be observed in the future. TESS_orbit_times.csv should be updated when available.
2/39: [ 29  69  96 103 104 105 106]
	Sector 29 doesn't overlap with 

In [48]:
spt_flares_df.head()

,ra,dec,obsid,mjd,flare_time_mjd,source_sectors,flare_sector
0,7.60272,-62.60104,136913663,59338.648877,59338.648877,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",None
1,321.16033,-64.00349,144824912,59430.214259,59430.214259,"[1, 27, 67, 94, 95, 101, 102, 103, 104]",None
2,14.22833,-51.87476,146323155,59447.555035,59447.555035,"[29, 69, 96, 103, 104, 105, 106]",None
3,326.12035,-60.97795,147124390,59456.828588,59456.828588,"[1, 27, 28, 67, 68, 94, 95, 101, 102, 103, 105]",None
4,334.78896,-42.58684,147748888,59464.056574,59464.056574,"[1, 68, 95, 105, 106]",None


In [49]:
spt_flares_df[spt_flares_df['flare_sector'].astype(bool)]

,ra,dec,obsid,mjd,flare_time_mjd,source_sectors,flare_sector
31,59.92104,-58.67446,256943755,60727.886053,60727.886053,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",89


# Format final dataframe
Now we will format to our desired data structure and fill in all relevant data, some of which may need to be filled by hand. <span style= "color:green; font-weight:bold;">This section is VERY manual for the moment</span>

<span style= "color:red; font-weight:bold;"> Before moving forward ensure that you have or can get the gaia dr3 id for these flares. This is the central identifier we will use. </span>

If you <span style= "font-weight:bold;">do not have gaia dr3 ids</span>, stop here as you are likely simply using this notebook to <span style= "font-weight:bold;">check if there are ANY overlaps</span>. 

If you <span style= "font-weight:bold;">do have gaia dr3 ids</span> input them manually if necessary.

We will produce 2 dataframes as follows:

<span style= "font-weight: bold;">spt_tess_candidates</span> JUST FOR flares with coinciding TESS observation and <span style= "font-weight: bold;">spt_flare_sources</span> FOR ALL SPT flare stars by Gaia dr3 id. Both with columns:
- 'gaia_dr3_id', 'TICID',
- 'source_ra', 'source_dec', 'flare_time_mjd',
- 'source_sectors', 'flare_sector',
- 'parallax', 'parallax_err'

In [59]:
spt_flare_sources = pd.DataFrame(columns=['gaia_dr3_id','TICID','source_ra','source_dec','flare_time_mjd','source_sectors','flare_sector','parallax','parallax_err'])
spt_flares_df.columns

Index(['ra', 'dec', 'obsid', 'mjd', 'flare_time_mjd', 'source_sectors',
       'flare_sector'],
      dtype='object')

Copy relevant columns

In [60]:
spt_flare_sources['source_ra'] = spt_flares_df[ra_col_name]
spt_flare_sources['source_dec'] = spt_flares_df[dec_col_name]
spt_flare_sources['flare_time_mjd'] = spt_flares_df['flare_time_mjd']
spt_flare_sources['source_sectors'] = spt_flares_df['source_sectors']
spt_flare_sources['flare_sector'] = spt_flares_df['flare_sector']

# Copy gaia_dr3_id, parallax, and parallax_rr if given otherwise grab below by (source_ra,source_dec)
# spt_flare_sources['gaia_dr3_id'] = spt_flares_df['dr3_source_id'].apply(lambda x: f"Gaia DR3 {int(x)}")
# spt_flare_sources['parallax'] = spt_flares_df['parallax']
# spt_flare_sources['parallax_err'] = spt_flares_df['parallax_err']

spt_flare_sources.head()

,gaia_dr3_id,TICID,source_ra,source_dec,flare_time_mjd,source_sectors,flare_sector,parallax,parallax_err
0,NaN,NaN,7.60272,-62.60104,59338.648877,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",None,NaN,NaN
1,NaN,NaN,321.16033,-64.00349,59430.214259,"[1, 27, 67, 94, 95, 101, 102, 103, 104]",None,NaN,NaN
2,NaN,NaN,14.22833,-51.87476,59447.555035,"[29, 69, 96, 103, 104, 105, 106]",None,NaN,NaN
3,NaN,NaN,326.12035,-60.97795,59456.828588,"[1, 27, 28, 67, 68, 94, 95, 101, 102, 103, 105]",None,NaN,NaN
4,NaN,NaN,334.78896,-42.58684,59464.056574,"[1, 68, 95, 105, 106]",None,NaN,NaN


Fill in remaining columns using astroquery

In [61]:
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
# from astropy.table import Table, vstack
from tqdm import tqdm
tqdm.pandas()

Find Gaia dr3 id using (ra,dec) in [Gaia Archive](https://gea.esac.esa.int/archive/)

Find parallax and parallax_err using gaia_dr3_id in [Gaia Archive](https://gea.esac.esa.int/archive/)

Find TICID using gaia_dr3_id in [SIMBAD](https://simbad.cds.unistra.fr/simbad/sim-fid)


In [62]:
def get_gaia_data(ra, dec, radius= 5 * u.arcsec): # dr3_id, parallax, parallax_err
    target = SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='icrs')

    # ??? Should there be a way to batch this into 1 query instead of 1 for each?
    query = f"""
    SELECT * 
    FROM gaiadr3.gaia_source 
    WHERE 1=CONTAINS(
        POINT('ICRS', ra, dec),
        CIRCLE('ICRS', {target.ra.degree}, {target.dec.degree}, {radius.to(u.deg).value})
    )
    """
    job = Gaia.launch_job_async(query)
    result = job.get_results()
    if len(result) > 0:
        # Select closes one
        coords = SkyCoord(ra=result['ra'], dec=result['dec'], unit='deg')
        separations = target.separation(coords)
        min_idx = separations.argmin()
        best = result[min_idx]

        # If Gaia DR3 IDs are missing then I likely have spt ra and dec to, so for consistency we will grab the gaia source_ra and source_dec
        return f"Gaia DR3 {int(best['SOURCE_ID'])}", best['parallax'], best['parallax_error'], best['ra'], best['dec']
    else:
        print(f"\t\tGaia query returned no DR3 ID for search radius < {radius.to(u.arcsec)}.")
        return None, None, None, None, None

# def get_parallax(gaia_dr3_ids):
#     id_list = ",".join(map(str, gaia_dr3_ids))
#     query = f"""
#     SELECT source_id, parallax, parallax_error
#     FROM gaiadr3.gaia_source
#     WHERE source_id IN ({id_list})
#     """
#     job = Gaia.launch_job_async(query)
#     return job.get_results().to_pandas().rename(columns={
#         'SOURCE_ID': 'gaia_dr3_id',
#         'parallax_error': 'parallax_err'
#     })

def get_TICID(gaia_dr3_id):
    table = Simbad.query_objectids(gaia_dr3_id)
    if table is not None:
        string = table['ID'].astype(str)
        table.replace_column('ID', string)
        data = table['ID'].data
        array = np.flatnonzero(np.core.defchararray.find(data, 'TIC') != -1)
        if len(array) == 1:
            return data[array[0]]
        else:
            print(f"\t\tSIMBAD query returned no TICID. ({gaia_dr3_id})")
            return None
    else:
        print(f"\t\tSIMBAD query could not find Gaia DR3 ID. ({gaia_dr3_id})")
        return None    

def fill_gaia_data(row):
    gaia_dr3_id = row['gaia_dr3_id']
    if not pd.isnull(gaia_dr3_id):
        print(f'\tGaia DR3 ID already filled: {gaia_dr3_id}')
        return row
    
    radius = 5
    while pd.isnull(gaia_dr3_id) and radius <= 20:
        source_ra, source_dec = row['source_ra'], row['source_dec']
        res = get_gaia_data(source_ra, source_dec, radius * u.arcsec)
        if res[0] is not None: 
            row['gaia_dr3_id'], row['parallax'], row['parallax_err'], row['source_ra'], row['source_dec'] = res
            gaia_dr3_id = row['gaia_dr3_id']
            print(f'\tGaia DR3 ID found w/ search radius {radius*u.arcsec}')
        radius += 5
    return row

# def fill_parallax(df):
#     ids_missing_parallax = df.loc[df['parallax'].isnull(), 'gaia_dr3_id'].dropna().astype(str).tolist()
#     if not ids_missing_parallax:
#         print("\tAll parallaxes already filled.")
#         return df
#     parallaxes = get_parallax(ids_missing_parallax)
    
#     df = df.set_index('gaia_dr3_id')
#     parallaxes = parallaxes.set_index('gaia_dr3_id')
#     df.update(parallaxes[['parallax', 'parallax_err']])

#     return df.reset_index()

def fill_TICID(row):
    gaia_dr3_id = row['gaia_dr3_id']
    if pd.isnull(gaia_dr3_id): 
        print("\tCouldn't find TICID: missing Gaia DR3 ID.")
        return row
    
    TICID = row['TICID']
    if pd.isnull(TICID):
        row['TICID'] = get_TICID(gaia_dr3_id)
    return row

def fill_rows(df):
    missing_dr3_ids = df.loc[df['gaia_dr3_id'].isnull()]
    if not missing_dr3_ids.empty:
        print('Filling Gaia DR3 IDs')
        df = df.progress_apply(fill_gaia_data, axis= 1)
    
    # print('Filling Parallax')
    # df = fill_parallax(df)

    missing_TICIDs = df.loc[df['TICID'].isnull()]
    if not missing_TICIDs.empty:
        print('Filling TICIDs')
        df = df.progress_apply(fill_TICID, axis= 1)

    return df

In [63]:
spt_flare_sources = fill_rows(spt_flare_sources)
spt_flare_sources['flare_sector'] = spt_flare_sources['flare_sector'].apply(lambda x: int(x) if pd.notnull(x) else None).astype('Int64')
spt_flare_sources

Filling Gaia DR3 IDs


  5%|▌         | 2/39 [00:02<00:52,  1.42s/it]

INFO: Query finished. [astroquery.utils.tap.core]
	Gaia DR3 ID found w/ search radius 5.0 arcsec


  8%|▊         | 3/39 [00:05<01:12,  2.03s/it]

INFO: Query finished. [astroquery.utils.tap.core]
	Gaia DR3 ID found w/ search radius 5.0 arcsec
INFO: Query finished. [astroquery.utils.tap.core]
		Gaia query returned no DR3 ID for search radius < 5.0 arcsec.


...

 79%|███████▉  | 31/39 [00:00<00:00, 81.79it/s]WARNING: BlankResponseWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR3 6466063063582742400 [astroquery.simbad.core]


		SIMBAD query could not find Gaia DR3 ID. (Gaia DR3 6451858850821154816)
		SIMBAD query could not find Gaia DR3 ID. (Gaia DR3 6482728567483006720)
		SIMBAD query returned no TICID. (Gaia DR3 4850405974392311296)
		SIMBAD query could not find Gaia DR3 ID. (Gaia DR3 6452221724016066432)
		SIMBAD query returned no TICID. (Gaia DR3 5554191685019290368)
		SIMBAD query returned no TICID. (Gaia DR3 6396977140235008384)
		SIMBAD query could not find Gaia DR3 ID. (Gaia DR3 6466063063582742400)


100%|██████████| 39/39 [00:00<00:00, 82.19it/s]

		SIMBAD query could not find Gaia DR3 ID. (Gaia DR3 4735509380111661568)


,gaia_dr3_id,TICID,source_ra,source_dec,flare_time_mjd,source_sectors,flare_sector,parallax,parallax_err
0,Gaia DR3 4900997562402606208,TIC 231914260,7.605676,-62.601319,59338.648877,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",<NA>,22.543941,0.016643
1,Gaia DR3 6403279953202667648,None,321.162121,-64.004277,59430.214259,"[1, 27, 67, 94, 95, 101, 102, 103, 104]",<NA>,0.37438,0.323956
2,Gaia DR3 4928347433108236416,TIC 322054057,14.231773,-51.875465,59447.555035,"[29, 69, 96, 103, 104, 105, 106]",<NA>,31.829892,0.87055
3,Gaia DR3 6409848126430574592,TIC 406277015,326.125892,-60.977887,59456.828588,"[1, 27, 28, 67, 68, 94, 95, 101, 102, 103, 105]",<NA>,21.545074,0.014095
4,Gaia DR3 6569006626440192512,TIC 92219075,334.792293,-42.587543,59464.056574,"[1, 68, 95, 105, 106]",<NA>,5.297292,0.019144
5,Gaia DR3 6528936333914427392,None,351.312570,-44.729827,59499.039630,"[2, 28, 29, 68, 69, 96, 103, 105, 106]",<NA>,1.737275,0.018987
6,Gaia DR3 4726387007013852672,TIC 201883033,39.754735,-58.187066,59507.784282,"[2, 3, 30, 69, 96, 97]",<NA>,--,--
7,Gaia DR3 4803409475871194624,TIC 151587650,88.363992,-43.558948,59588.034167,"[5, 6, 32, 33, 98]",<NA>,5.380951,0.015721
8,Gaia DR3 4876684474095606400,TIC 675445,73.706095,-29.561672,59596.503912,"[5, 32, 98, 106, 107]",<NA>,5.216504,0.013771
9,Gaia DR3 5575899652327731328,None,101.122969,-39.837365,59611.668576,"[6, 7, 33, 34, 61, 87, 88]",<NA>,0.602692,0.040149


In [64]:
spt_tess_candidates = spt_flare_sources[spt_flare_sources['flare_sector'].apply(lambda x: pd.notnull(x))].reset_index(drop= True)
spt_tess_candidates

,gaia_dr3_id,TICID,source_ra,source_dec,flare_time_mjd,source_sectors,flare_sector,parallax,parallax_err
0,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,60727.886053,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",89,2.590692,0.010606


In [65]:
spt_flare_sources = (
    spt_flare_sources
    .groupby('gaia_dr3_id', dropna=False)
    .agg({
        'TICID': 'first',
        'source_ra': 'first',
        'source_dec': 'first',
        'source_sectors': 'first',
        'parallax': 'first',
        'parallax_err': 'first',
        'flare_time_mjd': lambda x: list(x),
        'flare_sector': lambda x: list(x),
    })
    .reset_index()
)
spt_flare_sources = spt_flare_sources.rename(columns={'flare_sector': 'flare_sectors'})
spt_flare_sources

,gaia_dr3_id,TICID,source_ra,source_dec,source_sectors,parallax,parallax_err,flare_time_mjd,flare_sectors
0,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",2.590692,0.010606,[60727.88605324074],[89]
1,Gaia DR3 4693488004202579328,None,36.941870,-69.525577,"[1, 2, 27, 28, 29, 67, 68, 69, 94, 95, 96]",--,--,[60067.53914351852],[<NA>]
2,Gaia DR3 4708842443566593536,TIC 234509307,12.928508,-64.954608,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",2.349151,0.01391,[60239.64171296296],[<NA>]
3,Gaia DR3 4723685987981062656,TIC 207176480,47.108692,-59.373644,"[2, 3, 29, 30, 69, 96, 97]",3.312833,0.023505,[59739.27402777778],[<NA>]
4,Gaia DR3 4726387007013852672,TIC 201883033,39.754735,-58.187066,"[2, 3, 30, 69, 96, 97]",--,--,[59507.7842824074],[<NA>]
5,Gaia DR3 4735509380111661568,None,48.798162,-51.741826,"[3, 4, 29, 30, 31, 69, 97]",1.275087,0.01397,[60819.678240740745],[<NA>]
6,Gaia DR3 4742040410461492096,TIC 166852312,40.445863,-52.997951,"[2, 3, 29, 30, 69, 96, 97]",22.937051,0.04588,[60798.126793981486],[<NA>]
7,Gaia DR3 4743518532046419072,TIC 231061242,31.147099,-54.881795,"[2, 3, 29, 30, 69, 96, 97, 104, 105]",21.831385,0.072338,[60128.992731481485],[<NA>]
8,Gaia DR3 4743636248509810944,TIC 231068332,32.533261,-54.511077,"[2, 3, 29, 30, 69, 96, 97, 104, 105]",4.834732,0.03874,[59839.0685300926],[<NA>]
9,Gaia DR3 4745195905754074496,TIC 166808151,39.216167,-52.051002,"[2, 3, 29, 30, 69, 96, 97, 105]",25.74719,0.019815,[60248.084502314814],[<NA>]


In [66]:
# spt_tess_candidates.to_pickle('..\data\spt_tess_candidates.pkl')
# spt_flare_sources.to_pickle('..\data\spt_flare_sources.pkl')

# Save Data

### `spt_tess_candidates.pkl`

In [77]:
print(spt_tess_candidates.columns)
spt_tess_candidates

Index(['gaia_dr3_id', 'TICID', 'source_ra', 'source_dec', 'flare_time_mjd',
       'source_sectors', 'flare_sector', 'parallax', 'parallax_err'],
      dtype='object')


,gaia_dr3_id,TICID,source_ra,source_dec,flare_time_mjd,source_sectors,flare_sector,parallax,parallax_err
0,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,60727.886053,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",89,2.590692,0.010606


In [78]:
spt_tess_candidates_old = pd.read_pickle('..\data\spt_tess_candidates.pkl')
print(spt_tess_candidates_old.columns)
spt_tess_candidates_old

Index(['gaia_dr3_id', 'TICID', 'source_ra', 'source_dec', 'flare_time_mjd',
       'source_sectors', 'flare_sector', 'parallax', 'parallax_err'],
      dtype='object')


,gaia_dr3_id,TICID,source_ra,source_dec,flare_time_mjd,source_sectors,flare_sector,parallax,parallax_err
0,Gaia DR3 6388014157668558080,TIC 229807000,352.241030,-68.043088,59067.630000,"[1, 27, 28, 67, 68, 94, 95, 102, 103, 104]",28,21.814973,0.016096
1,Gaia DR3 6567445491726233216,TIC 279255411,331.540648,-46.063980,59068.280000,"[28, 68, 105, 106]",28,2.686802,0.032307
2,Gaia DR3 4727920241619206912,TIC 207138379,43.883166,-57.047891,59110.270000,"[2, 3, 29, 30, 69, 96, 97]",29,21.889704,0.021013
3,Gaia DR3 4954453580066220800,TIC 401838575,26.133623,-46.075694,59131.780000,"[2, 3, 29, 30, 69, 96, 103, 104, 105, 106]",30,25.949235,0.045132
4,Gaia DR3 6463748969563201408,TIC 79394645,319.964017,-53.449742,60132.993600,"[1, 27, 67, 94, 95, 104, 105]",67,33.234634,0.055061
5,Gaia DR3 6388014157668558080,TIC 229807000,352.241030,-68.043088,60140.479757,"[1, 27, 28, 67, 68, 94, 95, 102, 103, 104]",67,21.814973,0.016096
6,Gaia DR3 6463546350186454144,TIC 219317273,320.183645,-54.633106,60142.124387,"[1, 27, 67, 94, 95, 102, 104, 105]",67,4.996734,0.014739
7,Gaia DR3 4904142818493218304,None,13.575856,-59.422393,60185.975012,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104, 105]",69,16.088721,0.018403
8,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,60727.886053,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",89,2.590692,0.010606


In [69]:
spt_tess_candidates_new = pd.concat([spt_tess_candidates_old, spt_tess_candidates]).reset_index(drop= True)
spt_tess_candidates_new

,gaia_dr3_id,TICID,source_ra,source_dec,flare_time_mjd,source_sectors,flare_sector,parallax,parallax_err
0,Gaia DR3 6388014157668558080,TIC 229807000,352.241030,-68.043088,59067.630000,"[1, 27, 28, 67, 68, 94, 95, 102, 103, 104]",28,21.814973,0.016096
1,Gaia DR3 6567445491726233216,TIC 279255411,331.540648,-46.063980,59068.280000,"[28, 68, 105, 106]",28,2.686802,0.032307
2,Gaia DR3 4727920241619206912,TIC 207138379,43.883166,-57.047891,59110.270000,"[2, 3, 29, 30, 69, 96, 97]",29,21.889704,0.021013
3,Gaia DR3 4954453580066220800,TIC 401838575,26.133623,-46.075694,59131.780000,"[2, 3, 29, 30, 69, 96, 103, 104, 105, 106]",30,25.949235,0.045132
4,Gaia DR3 6463748969563201408,TIC 79394645,319.964017,-53.449742,60132.993600,"[1, 27, 67, 94, 95, 104, 105]",67,33.234634,0.055061
5,Gaia DR3 6388014157668558080,TIC 229807000,352.241030,-68.043088,60140.479757,"[1, 27, 28, 67, 68, 94, 95, 102, 103, 104]",67,21.814973,0.016096
6,Gaia DR3 6463546350186454144,TIC 219317273,320.183645,-54.633106,60142.124387,"[1, 27, 67, 94, 95, 102, 104, 105]",67,4.996734,0.014739
7,Gaia DR3 4904142818493218304,None,13.575856,-59.422393,60185.975012,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104, 105]",69,16.088721,0.018403
8,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,60727.886053,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",89,2.590692,0.010606


In [70]:
spt_tess_candidates_new.to_pickle('..\data\spt_tess_candidates.pkl')

### `spt_flare_sources.pkl`

In [71]:
print(len(spt_flare_sources))
spt_flare_sources.head()

39


,gaia_dr3_id,TICID,source_ra,source_dec,source_sectors,parallax,parallax_err,flare_time_mjd,flare_sectors
0,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",2.590692,0.010606,[60727.88605324074],[89]
1,Gaia DR3 4693488004202579328,None,36.941870,-69.525577,"[1, 2, 27, 28, 29, 67, 68, 69, 94, 95, 96]",--,--,[60067.53914351852],[<NA>]
2,Gaia DR3 4708842443566593536,TIC 234509307,12.928508,-64.954608,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",2.349151,0.01391,[60239.64171296296],[<NA>]
3,Gaia DR3 4723685987981062656,TIC 207176480,47.108692,-59.373644,"[2, 3, 29, 30, 69, 96, 97]",3.312833,0.023505,[59739.27402777778],[<NA>]
4,Gaia DR3 4726387007013852672,TIC 201883033,39.754735,-58.187066,"[2, 3, 30, 69, 96, 97]",--,--,[59507.7842824074],[<NA>]


In [72]:
spt_flare_sources_old = pd.read_pickle('..\data\spt_flare_sources.pkl')
print(len(spt_flare_sources_old))
print(spt_flare_sources_old.columns)
spt_flare_sources_old.head()

79
Index(['gaia_dr3_id', 'TICID', 'source_ra', 'source_dec', 'source_sectors',
       'parallax', 'parallax_err', 'flare_time_mjd', 'flare_sectors'],
      dtype='object')


,gaia_dr3_id,TICID,source_ra,source_dec,source_sectors,parallax,parallax_err,flare_time_mjd,flare_sectors
0,Gaia DR3 4691426694779173376,TIC 52256020,20.798151,-69.360675,"[1, 2, 27, 28, 29, 67, 68, 69, 94, 95, 96]",22.462200,0.077673,[58761.09],[<NA>]
1,Gaia DR3 4697144528904715008,None,39.630969,-64.459519,"[1, 2, 3, 28, 29, 30, 68, 69, 95]",1.056141,0.010487,[58576.15],[<NA>]
2,Gaia DR3 4708842443566593536,TIC 234509307,12.928508,-64.954608,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",2.349151,0.013910,"[59345.82, 59854.89, 60226.257881944446, 60239...","[<NA>, <NA>, <NA>, <NA>]"
3,Gaia DR3 4710627809932009344,TIC 234547938,16.828467,-61.775339,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",10.665785,0.053739,"[59788.74, 60830.46326388889]","[<NA>, <NA>]"
4,Gaia DR3 4713771622913507328,TIC 382552502,35.685243,-60.379948,"[3, 28, 30, 68, 69, 96, 97]",35.519346,0.019780,"[58798.28, 59710.62]","[<NA>, <NA>]"


In [73]:
spt_flare_sources_new = pd.concat([spt_flare_sources_old, spt_flare_sources]).reset_index(drop= True)
print(len(spt_flare_sources_new))
spt_flare_sources_new

118


,gaia_dr3_id,TICID,source_ra,source_dec,source_sectors,parallax,parallax_err,flare_time_mjd,flare_sectors
0,Gaia DR3 4691426694779173376,TIC 52256020,20.798151,-69.360675,"[1, 2, 27, 28, 29, 67, 68, 69, 94, 95, 96]",22.4622,0.077673,[58761.09],[<NA>]
1,Gaia DR3 4697144528904715008,None,39.630969,-64.459519,"[1, 2, 3, 28, 29, 30, 68, 69, 95]",1.056141,0.010487,[58576.15],[<NA>]
2,Gaia DR3 4708842443566593536,TIC 234509307,12.928508,-64.954608,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",2.349151,0.01391,"[59345.82, 59854.89, 60226.257881944446, 60239...","[<NA>, <NA>, <NA>, <NA>]"
3,Gaia DR3 4710627809932009344,TIC 234547938,16.828467,-61.775339,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",10.665785,0.053739,"[59788.74, 60830.46326388889]","[<NA>, <NA>]"
4,Gaia DR3 4713771622913507328,TIC 382552502,35.685243,-60.379948,"[3, 28, 30, 68, 69, 96, 97]",35.519346,0.01978,"[58798.28, 59710.62]","[<NA>, <NA>]"
...,...,...,...,...,...,...,...,...,...
113,Gaia DR3 6528936333914427392,None,351.312570,-44.729827,"[2, 28, 29, 68, 69, 96, 103, 105, 106]",1.737275,0.018987,[59499.03962962963],[<NA>]
114,Gaia DR3 6559052816392332288,TIC 139993838,328.601605,-49.944783,"[1, 28, 68, 95, 104, 105]",5.11379,0.013262,[60190.687951388885],[<NA>]
115,Gaia DR3 6560463421092689024,TIC 279255939,331.686357,-47.737493,"[1, 28, 68, 95, 105]",2.836155,0.027671,[59842.17087962963],[<NA>]
116,Gaia DR3 6561698791124472960,TIC 389726435,329.657411,-45.743346,"[1, 28, 68, 95, 105, 106]",3.837477,0.023148,[60068.95028935185],[<NA>]


In [74]:
spt_flare_sources_new = (
    spt_flare_sources_new
    .groupby('gaia_dr3_id', dropna=False)
    .agg({
        'TICID': 'first',
        'source_ra': 'first',
        'source_dec': 'first',
        'source_sectors': 'first',
        'parallax': 'first',
        'parallax_err': 'first',
        'flare_time_mjd': lambda x: sum(x, []),
        'flare_sectors': lambda x: sum(x, []),
    })
    .reset_index()
)
print(len(spt_flare_sources_new))
spt_flare_sources_new

96


,gaia_dr3_id,TICID,source_ra,source_dec,source_sectors,parallax,parallax_err,flare_time_mjd,flare_sectors
0,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",2.590692,0.010606,[60727.88605324074],[89]
1,Gaia DR3 4691426694779173376,TIC 52256020,20.798151,-69.360675,"[1, 2, 27, 28, 29, 67, 68, 69, 94, 95, 96]",22.4622,0.077673,[58761.09],[<NA>]
2,Gaia DR3 4693488004202579328,None,36.941870,-69.525577,"[1, 2, 27, 28, 29, 67, 68, 69, 94, 95, 96]",--,--,[60067.53914351852],[<NA>]
3,Gaia DR3 4697144528904715008,None,39.630969,-64.459519,"[1, 2, 3, 28, 29, 30, 68, 69, 95]",1.056141,0.010487,[58576.15],[<NA>]
4,Gaia DR3 4708842443566593536,TIC 234509307,12.928508,-64.954608,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104]",2.349151,0.01391,"[59345.82, 59854.89, 60226.257881944446, 60239...","[<NA>, <NA>, <NA>, <NA>, <NA>]"
...,...,...,...,...,...,...,...,...,...
91,Gaia DR3 6561698791124472960,TIC 389726435,329.657411,-45.743346,"[1, 28, 68, 95, 105, 106]",3.837477,0.023148,"[59877.83, 60068.95030092593, 60068.95028935185]","[<NA>, <NA>, <NA>]"
92,Gaia DR3 6567445491726233216,TIC 279255411,331.540648,-46.063980,"[28, 68, 105, 106]",2.686802,0.032307,[59068.28],[28]
93,Gaia DR3 6569006626440192512,TIC 92219075,334.792293,-42.587543,"[1, 68, 95, 105, 106]",5.297292,0.019144,"[59464.06, 59464.05657407407]","[<NA>, <NA>]"
94,Gaia DR3 6577998398172195840,TIC 147250411,323.940164,-42.309798,"[1, 28, 68, 95, 105, 106]",20.524045,0.039617,[59408.63],[<NA>]


In [75]:
spt_flare_sources_new.to_pickle('..\data\spt_flare_sources.pkl')

In [81]:
import math
temp = spt_flare_sources_new[spt_flare_sources_new['flare_sectors'].apply(lambda x: sum(not pd.isna(y) for y in x) != 0)].reset_index(drop= True)
temp

,gaia_dr3_id,TICID,source_ra,source_dec,source_sectors,parallax,parallax_err,flare_time_mjd,flare_sectors
0,Gaia DR3 4682624554442312576,TIC 198006444,59.919241,-58.675042,"[2, 3, 4, 29, 30, 31, 35, 62, 65, 69, 89, 96, 97]",2.590692,0.010606,[60727.88605324074],[89]
1,Gaia DR3 4727920241619206912,TIC 207138379,43.883166,-57.047891,"[2, 3, 29, 30, 69, 96, 97]",21.889704,0.021013,[59110.27],[29]
2,Gaia DR3 4904142818493218304,None,13.575856,-59.422393,"[1, 2, 28, 29, 68, 69, 95, 96, 103, 104, 105]",16.088721,0.018403,[60185.975011574075],[69]
3,Gaia DR3 4954453580066220800,TIC 401838575,26.133623,-46.075694,"[2, 3, 29, 30, 69, 96, 103, 104, 105, 106]",25.949235,0.045132,"[59131.78, 60125.126597222225]","[30, <NA>]"
4,Gaia DR3 6388014157668558080,TIC 229807000,352.241030,-68.043088,"[1, 27, 28, 67, 68, 94, 95, 102, 103, 104]",21.814973,0.016096,"[59067.63, 60140.47975694444]","[28, 67]"
5,Gaia DR3 6463546350186454144,TIC 219317273,320.183645,-54.633106,"[1, 27, 67, 94, 95, 102, 104, 105]",4.996734,0.014739,"[58755.38, 59003.11, 59005.5, 59096.39, 59308....","[<NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA..."
6,Gaia DR3 6463748969563201408,TIC 79394645,319.964017,-53.449742,"[1, 27, 67, 94, 95, 104, 105]",33.234634,0.055061,[60132.99359953704],[67]
7,Gaia DR3 6567445491726233216,TIC 279255411,331.540648,-46.063980,"[28, 68, 105, 106]",2.686802,0.032307,[59068.28],[28]
